In [2]:
import pandas as pd
import spacy, re
import numpy as np

# Exemple de lemmatisation

In [1]:
inputText =\
"""
Bonjour bonjour

Lors de notre point mensuel sur la CSS je m’aperçois que sur le mois de mars nous avons énormément de tardifs qui ont été réglés sur mars en comparaison au mois de février. Je ne m’explique pas ce phénomène surtout que de ce que j’avais compris les non règlements du fait du pb avec OXANTIS ne concernaient que 5M€ au total et pour des soins 2019 et que cela devait finalement être récupéré sur le mois d’avril. De mon côté je ne pense pas avoir vu d’alerte sur des factures datant de 2015 et après qui n’auraient pas été réglées. D’où « sortent » ces règlements ?

Je n’ai surement pas tout suivi dans le détail mais je trouve que les tardifs sont très élevés et il me sera demandé une explication par le Fonds CSS. Il est nécessaire je pense d’approfondir ce sujet car cela pourrait avoir des répercussions importantes sur notre définitif au niveau de la mutuelle si ce phénomène est généralisé

   merci côté stats de me donner des précisions sur ce constat pour la CSS et si ceci est également constaté sur l’ensemble du portefeuille HM.
	Et en // merci à Nadège de regarder si tout est OK coté intégration dans DECIBEL

D’avance je vous remercie pour toutes les précisions que vous pourrez m’apporter et je vous souhaite une bonne journée

"""

In [2]:
#fr_dep_news_trf, fr_core_news_md
lemmatizer = spacy.load("fr_core_news_md", disable = ['parser', 'attribute_ruler', 'ner'])
print("Pipeline:", lemmatizer.pipe_names)

Pipeline: ['tok2vec', 'morphologizer', 'lemmatizer']


In [14]:
doc = lemmatizer(inputText)  # Create a Doc from raw text
words_lemmas_list = [token.lemma_.lower() for token in doc if not token.is_stop and re.match('^[a-z]+$', token.lemma_.lower()) is not None]
print(words_lemmas_list)

['bonjour', 'bonjour', 'point', 'mensuel', 'css', 'apercevoir', 'mois', 'mars', 'tardif', 'mars', 'comparaison', 'mois', 'explique', 'non', 'oxantis', 'concerner', 'total', 'soin', 'devoir', 'finalement', 'mois', 'avril', 'alerte', 'facture', 'dater', 'sortir', 'surement', 'suivre', 'trouve', 'tardif', 'demander', 'explication', 'fonds', 'css', 'approfondir', 'sujet', 'important', 'niveau', 'mutuel', 'stat', 'donner', 'constat', 'css', 'constater', 'ensemble', 'portefeuille', 'hm', 'regarder', 'ok', 'decibel', 'avance', 'remercier', 'pouvoir', 'apporter', 'souhaite', 'bon']


# Application au dataset

In [48]:
with open('firstdataset.csv','r', encoding='utf-16') as csvfile:
    data = csvfile.read()
with open('firstdataset2.csv','w', encoding='utf-16') as csvfile:
    csvfile.write(data.replace('\x00',''))

In [1]:
dataset1 = pd.read_csv('firstdataset2.csv',index_col = 0,sep='¤',quotechar='§',encoding='UTF-16', engine='python',header=0, skipinitialspace=True, skip_blank_lines = True)
print(dataset1.shape)
dataset1.head()

(3296, 3)


,demande_de_support,sender,text
0,0,plonquet nadège,1ère relecture gt conso yohan quand tu as le t...
1,0,plonquet nadège,accepter pour accepter la demande cliquez simp...
2,1,guillaume veronique,acces decibel bonjour pouvez-vous donner les a...
3,1,gueniot bernard,accès rec4 decibel sdw-rec4 hm dm ad restituti...
4,1,levisse xavier,actes indemnités hospitalières tu sais ce que ...


In [2]:
from preprocessing import WriteFinalDataset
WriteFinalDataset(inputFile='firstdataset2.csv',outputFile='finaldataset.csv')

Lecture de firstdataset2.csv ...
Traitement des données ...
[############################################################] 3296/3296


Ecriture de finaldataset.csv ...


In [ ]:
data = pd.read_csv('finaldataset.csv', index_col = 0, header=0, encoding='utf-16', dtype=int)
data
're' in data.columns

,relecture,gt,consommation,yohan,temps,dire,clair,mettre,decibel,evolution,...,reell,reellesup,rosup,rcsup,noyer,brosset,lavoisier,lepaumier,minuscule,demande_de_support
0,1,1,1,1,1,1,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291,0,0,0,0,0,0,0,2,0,0,...,1,1,1,1,0,0,0,0,0,0
3292,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
3293,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,0,0


In [7]:
from collections import Counter
Counter(data['demande_de_support'])

Counter({0: 2673, 1: 454, 2: 169})

## KMeans et apprentissage semi-supervisé

In [ ]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.style.use('ggplot')

In [13]:
X_unlabeled = data.loc[data["demande_de_support"] == 2].drop(columns=['demande_de_support'])
X = data.loc[data["demande_de_support"] != 2].drop(columns=['demande_de_support'])
y = pd.Series(data.loc[data["demande_de_support"] != 2]['demande_de_support'], dtype=int)

In [30]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
kmeans1 = KMeans(n_clusters=2)
kmeans1.fit(X_train, y_train)

y_pred = kmeans1.predict(X_test)
accuracy_score(y_test,y_pred)

0.7795527156549521

In [15]:
new_labels = kmeans1.predict(X_unlabeled)
newX = np.vstack((X, X_unlabeled))
newY = np.hstack((y, new_labels))

print(newX.shape, newY.shape)

(3296, 7522) (3296,)


## Construction d'un prédicteur

In [32]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.2, random_state=90210)

gridCV = GridSearchCV(SVC(),
    param_grid = {
        'C' : np.linspace(0.1,3,3),
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        #'degree' : [1, 2, 3, 4],
        'gamma' : ['scale', 'auto'] + list(np.linspace(0,2,3)),
        #'coef0' : np.linspace(-2,2,5),
        #'probability' : [True, False],
        #'class_weight' : ['balanced', None],
        #'decision_function_shape' : ['ovo', 'ovr']
    }
)

gridCV.fit(X_train, y_train)
gridCV.best_params_

C:\Users\lepau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lepau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lepau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py", line 226, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\lepau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-p